In [1]:
%use klaxon
%use lets-plot

import java.net.*
import kotlin.time.*
import jetbrains.letsPlot.scale.*


In [42]:
data class FftSample(
    val index: Long, 
    val binCount: Int, 
    val samplesCount: Int, 
    val sampleRate: Float, 
    val magnitude: List<Double>,
    val phase: List<Double>,
    val frequency: List<Double>,
    val time: Long
)
data class Result(val offset: Long, val value: FftSample)

val k = Klaxon()

val data = URL("http://localhost:6800/table/fft/last?interval=10s").openStream().reader().readLines()
.map { k.parse<Result>(it)!! }


In [48]:
val freqCutOff = 10 to 3000 // Hz
val timeShift = data.asSequence().map { it.value.time }.min() ?: 0
    
val table = data.asSequence().map {v -> 
        v.value.magnitude.asSequence()
            .zip(v.value.frequency.asSequence())
            .filter { it.second >= freqCutOff.first && it.second <= freqCutOff.second}
            .map { arrayOf(
                    (v.value.time - timeShift) / 10e6,
                    it.second, 
                    it.first
            )}
    }
    .flatten()
    .toList() // this gives a list of tuples [ (time, freq, value), ... , (time, freq, value)]

val dataFrame = mapOf(
    "time" to table.map { it[0] },
    "freq" to table.map { it[1] },
    "value" to table.map { it[2] }
)    

println("entries count = ${dataFrame.entries.first().value.size}")
println("distinct time values count = ${dataFrame["time"]!!.distinct().count()}")

entries count = 37366
distinct time values count = 1099


In [47]:

lets_plot(dataFrame) {x = "time"; y = "freq"; fill = "value"} + 
    ggsize(1000, 600) + 
    geom_tile() + 
    scale_fill_gradient(low = "light_green", high = "red")
    